# 3x3 assembly inputfile that: 

### Import module/Set up XS path/Create openMC model/Clean directory

In [1]:
import sys
import os 
import openmc
import numpy as np
import matplotlib.pyplot as plt
import openmc.mgxs as mgxs
import pandas as pd
from IPython.display import Image
import glob

### Pablo environnement

In [2]:
model=openmc.Model()

clean_file_list = ["model.xml", "materials.xml", "geometry.xml","plots.xml","geometry.xml","settings.xml","tallies.out","summary.h5","statepoint.50.h5"]
for file in clean_file_list :
    path_file = os.path.join(os.getcwd(), file)
    if os.path.exists(path_file):
        os.remove(path_file)

### Define Materials

In [3]:
uo2 = openmc.Material(name='uo2')
uo2.add_nuclide('U234', 6.11864E-06, 'ao')
uo2.add_nuclide('U235', 7.18132E-04, 'ao')
uo2.add_nuclide('U236', 3.29861E-06, 'ao')
uo2.add_nuclide('U238', 2.21546E-02, 'ao')
uo2.add_nuclide('O16', 4.57642E-02, 'ao')
uo2.set_density('g/cm3', 10.257 )


zirconium = openmc.Material(name="zirconium")
zirconium.add_nuclide('Zr90', 2.18865E-02, 'ao')
zirconium.add_nuclide('Zr91', 4.77292E-03, 'ao')
zirconium.add_nuclide('Zr92', 7.29551E-03, 'ao')
zirconium.add_nuclide('Zr94', 7.39335E-03, 'ao')
zirconium.add_nuclide('Zr96', 1.19110E-03, 'ao')
zirconium.add_nuclide('Sn112',4.68066E-06, 'ao')
zirconium.add_nuclide('Sn114', 3.18478E-06, 'ao')
zirconium.add_nuclide('Sn115', 1.64064E-06, 'ao')
zirconium.add_nuclide('Sn116', 7.01616E-05, 'ao')
zirconium.add_nuclide('Sn117', 3.70592E-05, 'ao')
zirconium.add_nuclide('Sn118', 1.16872E-04, 'ao')
zirconium.add_nuclide('Sn119', 4.14504E-05, 'ao')
zirconium.add_nuclide('Sn120', 1.57212E-04, 'ao')
zirconium.add_nuclide('Sn122', 2.23417E-05, 'ao')
zirconium.add_nuclide('Sn124', 2.79392E-05, 'ao')
zirconium.add_nuclide('Fe54', 8.68307E-06, 'ao')
zirconium.add_nuclide('Fe56', 1.36306E-04, 'ao')
zirconium.add_nuclide('Fe57', 3.14789E-06, 'ao')
zirconium.add_nuclide('Fe58', 4.18926E-07, 'ao')
zirconium.add_nuclide('Cr50', 3.30121E-06, 'ao')
zirconium.add_nuclide('Cr52', 6.36606E-05, 'ao')
zirconium.add_nuclide('Cr53', 7.21860E-06, 'ao')
zirconium.add_nuclide('Cr54', 1.79686E-06, 'ao')
zirconium.add_nuclide('Hf174', 3.54138E-09, 'ao')
zirconium.add_nuclide('Hf176', 1.16423E-07, 'ao')
zirconium.add_nuclide('Hf177', 4.11686E-07, 'ao')
zirconium.add_nuclide('Hf178', 6.03806E-07, 'ao')
zirconium.add_nuclide('Hf179', 3.01460E-07, 'ao')
zirconium.add_nuclide('Hf180', 7.76449E-07, 'ao')
zirconium.set_density('g/cm3',  6.56)



water = openmc.Material(name="water")
water.add_nuclide('H1',4.96224E-02, 'ao')
water.add_nuclide('O16', 2.48112E-02, 'ao')
water.add_nuclide('B10', 1.07070E-05, 'ao')
water.add_nuclide('B11', 4.30971E-05, 'ao')
water.set_density('g/cm3', 0.743)
water.add_s_alpha_beta('c_H_in_H2O')


materials_file = openmc.Materials([uo2, zirconium, water]) 


model.materials = openmc.Materials([uo2, zirconium, water]) 
#Perso path
#model.materials.cross_sections = '/home/pablo/internship/xs_for_input/cross_sections.xml'
#Mac path
model.materials.cross_sections = '/Users/pablogarcia44/repo/endfb-vii.1-hdf5/cross_sections.xml'

### Create pincell

In [4]:
fuel_outer_radius = openmc.ZCylinder(r=0.418)
clad_outer_radius = openmc.ZCylinder(r=0.475)
pitch = 1.26
rod_height=1
left = openmc.XPlane(-pitch/2, boundary_type='reflective')
right = openmc.XPlane(pitch/2, boundary_type='reflective')
bottom = openmc.YPlane(-pitch/2, boundary_type='reflective')
top = openmc.YPlane(pitch/2, boundary_type='reflective')
fuel_region = -fuel_outer_radius
clad_region = +fuel_outer_radius & -clad_outer_radius
water_region = +left & -right & +bottom & -top & +clad_outer_radius
fuel = openmc.Cell(name='fuel')
fuel.fill = uo2
fuel.region = fuel_region
fuel.temperature = 565
clad = openmc.Cell(name='clad')
clad.fill = zirconium
clad.region = clad_region
clad.temperature = 565
moderator = openmc.Cell(name='moderator')
moderator.fill = water
moderator.region = water_region 
moderator.temperature = 565
root_universe = openmc.Universe(cells=(fuel, clad, moderator))
model.geometry = openmc.Geometry(root_universe)


### Plot

model.materials.export_to_xml()
model.geometry.export_to_xml()
plot = openmc.Plot.from_geometry(model.geometry)
plot.pixels = (500, 500)
plot.width = (pitch_assembly+dr, pitch_assembly+dr)
plot.origin = (0., 0., 0)
plot.color_by = 'cell'
plots.append(plot)
model.plots=openmc.Plots(plots)
#plot.to_ipython_image(openmc_exec='openmc', cwd='/Users/pablogarcia44/repo/perso/openmc/17x17_assembly/vera')

### Choose settings

In [5]:
bounds = [-pitch/2, -pitch/2, -rod_height/2, pitch/2, +pitch/2, rod_height/2]
uniform_dist = openmc.stats.Box(bounds[:3], bounds[3:], only_fissionable=True)
source = openmc.IndependentSource(space=uniform_dist)
settings = openmc.Settings()
settings.source = source
settings = openmc.Settings()
settings.source = source
settings.batches = 30
settings.inactive = 20
settings.particles = 20000
settings.keff_trigger = {'type':'std_dev','threshold':0.00030}
settings.trigger_active = True
settings.trigger_max_batches = 50000
settings.output = {'tallies':True}
settings.temperature['method'] = 'interpolation'
settings_file = settings
model.settings = settings

### Define and choose energy group

In [6]:
choose_group=openmc.mgxs.GROUP_STRUCTURES['XMAS-172']
name_group='XMAS-172'

### Create MGXS tallies for mgxs directory

In [7]:
mgxs_lib_order0 = openmc.mgxs.Library(model.geometry)
groups = openmc.mgxs.EnergyGroups(choose_group)
mgxs_lib_order0.energy_groups = groups
mgxs_lib_order0.scatter_format = "legendre"
mgxs_lib_order0.mgxs_types = ['transport','nu-transport','absorption', 'nu-fission', 'fission' ,'chi', 'consistent scatter matrix', 'multiplicity matrix','total','consistent nu-scatter matrix']
mgxs_lib_order0.legendre_order = 0
mgxs_lib_order0.correction = None
mgxs_lib_order0.by_nuclide = False
mgxs_lib_order0.domain_type = 'cell'
mgxs_lib_order0.domains = model.geometry.get_all_material_cells().values()
mgxs_lib_order0.build_library()
mgxs_lib_order0.check_library_for_openmc_mgxs()

In [8]:
mgxs_lib_order1 = openmc.mgxs.Library(model.geometry)
groups = openmc.mgxs.EnergyGroups(choose_group)
mgxs_lib_order1.energy_groups = groups
mgxs_lib_order1.scatter_format = "legendre"
mgxs_lib_order1.mgxs_types = ['transport','nu-transport','absorption', 'nu-fission', 'fission' ,'chi', 'consistent scatter matrix', 'multiplicity matrix','total','consistent nu-scatter matrix']
mgxs_lib_order1.legendre_order = 1
# mgxs_lib_order3.correction = 'P0'
mgxs_lib_order1.by_nuclide = False
mgxs_lib_order1.domain_type = 'cell'
mgxs_lib_order1.domains = model.geometry.get_all_material_cells().values()
mgxs_lib_order1.build_library()
mgxs_lib_order1.check_library_for_openmc_mgxs()

/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mgxs/library.py:401: RuntimeWarning: The P0 correction will be ignored since the scattering order 1 is greater than zero
  warn(msg, RuntimeWarning)


In [9]:
mgxs_lib_order3 = openmc.mgxs.Library(model.geometry)
groups = openmc.mgxs.EnergyGroups(choose_group)
mgxs_lib_order3.energy_groups = groups
mgxs_lib_order3.scatter_format = "legendre"
mgxs_lib_order3.mgxs_types = ['transport','nu-transport','absorption', 'nu-fission', 'fission' ,'chi', 'consistent scatter matrix', 'multiplicity matrix','total','consistent nu-scatter matrix']
mgxs_lib_order3.legendre_order = 3
# mgxs_lib_order3.correction = 'P0'
mgxs_lib_order3.by_nuclide = False
mgxs_lib_order3.domain_type = 'cell'
mgxs_lib_order3.domains = model.geometry.get_all_material_cells().values()
mgxs_lib_order3.build_library()
mgxs_lib_order3.check_library_for_openmc_mgxs()

/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mgxs/library.py:401: RuntimeWarning: The P0 correction will be ignored since the scattering order 3 is greater than zero
  warn(msg, RuntimeWarning)


In [10]:
mgxs_lib_order5 = openmc.mgxs.Library(model.geometry)
groups = openmc.mgxs.EnergyGroups(choose_group)
mgxs_lib_order5.energy_groups = groups
mgxs_lib_order5.scatter_format = "legendre"
mgxs_lib_order5.mgxs_types = ['transport','nu-transport','absorption', 'nu-fission', 'fission' ,'chi', 'consistent scatter matrix', 'multiplicity matrix','total','consistent nu-scatter matrix']
mgxs_lib_order5.legendre_order = 5
# mgxs_lib_order3.correction = 'P0'
mgxs_lib_order5.by_nuclide = False
mgxs_lib_order5.domain_type = 'cell'
mgxs_lib_order5.domains = model.geometry.get_all_material_cells().values()
mgxs_lib_order5.build_library()
mgxs_lib_order5.check_library_for_openmc_mgxs()

/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mgxs/library.py:401: RuntimeWarning: The P0 correction will be ignored since the scattering order 5 is greater than zero
  warn(msg, RuntimeWarning)


In [11]:
mgxs_lib_order7 = openmc.mgxs.Library(model.geometry)
groups = openmc.mgxs.EnergyGroups(choose_group)
mgxs_lib_order7.energy_groups = groups
mgxs_lib_order7.scatter_format = "legendre"
mgxs_lib_order7.mgxs_types = ['transport','nu-transport','absorption', 'nu-fission', 'fission' ,'chi', 'consistent scatter matrix', 'multiplicity matrix','total','consistent nu-scatter matrix']
mgxs_lib_order7.legendre_order = 7
# mgxs_lib_order3.correction = 'P0'
mgxs_lib_order7.by_nuclide = False
mgxs_lib_order7.domain_type = 'cell'
mgxs_lib_order7.domains = model.geometry.get_all_material_cells().values()
mgxs_lib_order7.build_library()
mgxs_lib_order7.check_library_for_openmc_mgxs()

/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mgxs/library.py:401: RuntimeWarning: The P0 correction will be ignored since the scattering order 7 is greater than zero
  warn(msg, RuntimeWarning)


In [12]:
tallies = openmc.Tallies()
mgxs_lib_order0.add_to_tallies_file(tallies, merge=False)
mgxs_lib_order1.add_to_tallies_file(tallies, merge=False)
mgxs_lib_order3.add_to_tallies_file(tallies, merge=False)
mgxs_lib_order5.add_to_tallies_file(tallies, merge=False)
mgxs_lib_order7.add_to_tallies_file(tallies, merge=False)

model.tallies = tallies

### Create power tally

model.export_to_model_xml('model_iso.xml')

### Run OpenMC model

In [13]:
statepoint_filename = model.run()

/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=1.
  warn(msg, IDWarning)
/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=2.
  warn(msg, IDWarning)
/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=3.
  warn(msg, IDWarning)
/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=4.
  warn(msg, IDWarning)
/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=21.
  warn(msg, IDWarning)
/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=31.
  warn(msg, IDWarning)
/opt/anaconda3

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

In [14]:
ce_spfile = './statepoint_ce.h5'
os.rename(statepoint_filename, ce_spfile)
ce_sumfile = './summary_ce.h5'
os.rename('summary.h5', ce_sumfile)
sp = openmc.StatePoint(ce_spfile, autolink=False)
su = openmc.Summary(ce_sumfile)
sp.link_with_summary(su)

In [15]:
mgxs_lib_order0.load_from_statepoint(sp)
mgxs_lib_order1.load_from_statepoint(sp)
mgxs_lib_order3.load_from_statepoint(sp)
mgxs_lib_order5.load_from_statepoint(sp)
mgxs_lib_order7.load_from_statepoint(sp)

mgxs_file_order0 = mgxs_lib_order0.create_mg_library(xs_type='macro', xsdata_names=['fuel', 'clad', 'moderator'])
mgxs_file_order1 = mgxs_lib_order0.create_mg_library(xs_type='macro', xsdata_names=['fuel', 'clad', 'moderator'])
mgxs_file_order3 = mgxs_lib_order3.create_mg_library(xs_type='macro', xsdata_names=['fuel', 'clad', 'moderator'])
mgxs_file_order5 = mgxs_lib_order5.create_mg_library(xs_type='macro', xsdata_names=['fuel', 'clad', 'moderator'])
mgxs_file_order7 = mgxs_lib_order3.create_mg_library(xs_type='macro', xsdata_names=['fuel', 'clad', 'moderator'])



In [16]:
model_order0=openmc.Model()
mgxs_file_order0, materials_file, geometry_file = mgxs_lib_order0.create_mg_mode()

mgxs_file_order0.xsdatas[0].temperatures=[565.]
mgxs_file_order0.xsdatas[1].temperatures=[565.]
mgxs_file_order0.xsdatas[2].temperatures=[565.]


mgxs_file_order0.export_to_hdf5('mgxs_order0.h5')
materials_file.cross_sections = 'mgxs_order0.h5'
model_order0.materials = materials_file
model_order0.geometry = geometry_file
settings_file.energy_mode = 'multi-group'
model_order0.settings = settings_file

In [17]:
statepoint_filename0 = model_order0.run()

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

In [18]:
model_order1=openmc.Model()
mgxs_file_order1, materials_file, geometry_file = mgxs_lib_order1.create_mg_mode()

mgxs_file_order1.xsdatas[0].temperatures=[565.]
mgxs_file_order1.xsdatas[1].temperatures=[565.]
mgxs_file_order1.xsdatas[2].temperatures=[565.]
materials_file.cross_sections = 'mgxs_order1.h5'

mgxs_file_order1.export_to_hdf5('mgxs_order1.h5')
model_order1.materials = materials_file
model_order1.geometry = geometry_file
settings_file.energy_mode = 'multi-group'
model_order1.settings = settings_file

In [19]:
statepoint_filename1 = model_order1.run()

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

In [20]:
model_order3=openmc.Model()
mgxs_file_order3, materials_file, geometry_file = mgxs_lib_order3.create_mg_mode()

mgxs_file_order3.xsdatas[0].temperatures=[565.]
mgxs_file_order3.xsdatas[1].temperatures=[565.]
mgxs_file_order3.xsdatas[2].temperatures=[565.]
materials_file.cross_sections = 'mgxs_order3.h5'

mgxs_file_order3.export_to_hdf5('mgxs_order3.h5')
model_order3.materials = materials_file
model_order3.geometry = geometry_file
settings_file.energy_mode = 'multi-group'
model_order3.settings = settings_file

In [21]:
statepoint_filename3 = model_order3.run()

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

In [22]:
model_order5=openmc.Model()
mgxs_file_order5, materials_file, geometry_file = mgxs_lib_order5.create_mg_mode()

mgxs_file_order5.xsdatas[0].temperatures=[565.]
mgxs_file_order5.xsdatas[1].temperatures=[565.]
mgxs_file_order5.xsdatas[2].temperatures=[565.]
materials_file.cross_sections = 'mgxs_order5.h5'

mgxs_file_order5.export_to_hdf5('mgxs_order5.h5')
model_order5.materials = materials_file
model_order5.geometry = geometry_file
settings_file.energy_mode = 'multi-group'
model_order5.settings = settings_file

In [23]:
statepoint_filename5 = model_order5.run()

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

In [24]:
model_order7=openmc.Model()
mgxs_file_order7, materials_file, geometry_file = mgxs_lib_order7.create_mg_mode()

mgxs_file_order7.xsdatas[0].temperatures=[565.]
mgxs_file_order7.xsdatas[1].temperatures=[565.]
mgxs_file_order7.xsdatas[2].temperatures=[565.]
materials_file.cross_sections = 'mgxs_order7.h5'

mgxs_file_order7.export_to_hdf5('mgxs_order7.h5')
model_order7.materials = materials_file
model_order7.geometry = geometry_file
settings_file.energy_mode = 'multi-group'
model_order7.settings = settings_file

In [25]:
statepoint_filename7 = model_order7.run()

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################